In [97]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [12]:
# Initiate Selenium
options = Options
options.add_argument('--headless')
browser = webdriver.Chrome(executable_path = "D:\Program File\Python\AllEnv\envlatihan\Latihan\Selenium\chromedriver.exe", chrome_options=options)

d:\Program File\Python\AllEnv\envlatihan\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [ ]:
# url = "https://www.jamtangan.com/c/jam-tangan-pria?sort=best_selling&page=1"
# browser.get(url)

In [18]:
# soup = BeautifulSoup(browser.page_source, 'lxml')
# test = soup.find_all('div', {'class':'product p-1 md:p-2 w-1-2 sm:w-1-3 md:w-1-4 lg:w-1-5'})

In [34]:
# Create DataFrame to store many product links
hreflist = pd.DataFrame()

In [35]:
# Function to scrap product link, only use page 1-10 for each category
for category in ['pria','wanita','couple','unisex']:
  for num in range(1,11):
    try:
      url = f"https://www.jamtangan.com/c/jam-tangan-{category}?sort=best_selling&page={num}"
      browser.get(url)
      # page = requests.get(f"https://www.jamtangan.com/c/jam-tangan-{category}?sort=best_selling&page={num}")
      soup = BeautifulSoup(browser.page_source, 'lxml')
      test = soup.find_all('div', {'class':'product p-1 md:p-2 w-1-2 sm:w-1-3 md:w-1-4 lg:w-1-5'})
      for i in test:
        x = i.find('a', {'data-testid':"product-card-test"})
        # hreflist.append(x['href'])
        hreflist2 = pd.DataFrame({'cat':category, 'href':x['href']}, index=[0])
        hreflist = pd.concat([hreflist, hreflist2])
        # print(hreflist2)
    except:
      print('error')

In [36]:
hreflist.shape

(1377, 2)

In [44]:
hreflist2 = hreflist.drop_duplicates(subset='href')
len(hreflist2)

1286

In [51]:
# Function to scrap product attributes
def getfirst(soup1, url):
  test1 = soup1.find('ul', {'class':'text-xxs font-bold text-neutral-600 flex gap-1 leading-4'})
  cat = test1.find_all('li')[1].text
  price = soup1.find('div', {'class':'text-primary-1 text-xl font-black md:text-3xl'}).text
  jumlah = soup1.find('div', {'class':'p-4 border-b-1 border-neutral-800 md:border-neutral-900 text-neutral-300 w-full text-sm order-3 md:order-4'})
  jumlah1 = jumlah.find_all('div', {'class','flex items-center'})
  dilihat = jumlah1[0].find('div', {'class':'text-sm font-black'}).text
  dibeli = jumlah1[1].find('div', {'class':'text-sm font-black'}).text
  ulasan = soup1.find('div', {'class':'text-xxs text-neutral-600 ml-4 font-medium'}).text
  reward_point = soup1.find('div', {'class':'text-sm text-neutral-300 mr-4'}).text[15:]
  rating = soup1.find('p', {'class':'text-3xl md:text-5xl'}).text
  link = url
  testdict = {'price':price, 'cat':cat, 'dilihat':dilihat, 'dibeli':dibeli, 'ulasan':ulasan, 'reward_point':reward_point, 'link':link, 'rating':rating}

  test2 = soup1.find_all('div', {'class':'w-full col-span-12 md:col-span-6 mb-4 pr-2 text-sm spec-item'})
  for i in test2:
    keys = i.find('div', {'class':'font-black leading-6'}).text
    values = i.find_all('div', {'class':'leading-6'})[1:]
    if len(values) > 1:
      speclist = []
      for z in values:
        if z.text == '':
          continue
        else:
          speclist.append(z.text.strip().replace(u'\xa0', u' '))
          testdict[keys] = speclist
          # df[keys] = speclist
    else:
      value = values[0].text.strip().replace(u'\xa0', u' ')
      testdict[keys] = value
      # df[keys] = value
      
  return testdict

In [56]:
# Store product attributes in dataframe
df = pd.DataFrame()
for i, row in hreflist2.iterrows():
  url = f"https://www.jamtangan.com{row['href']}"
  browser.get(url)
  try:
    soup1 = BeautifulSoup(browser.page_source, 'lxml')
    testdict = getfirst(soup1, url)
    df1 = pd.json_normalize(testdict)
    df = pd.concat([df, df1])
  # else:
  except:
    print(f"error {url}")

error https://www.jamtangan.com/p/alexandre-christie-ac-9601-bbrba-couple-black-dial-black-stainless-steel-strap-459516
error https://www.jamtangan.com/p/tissot-t109.410.33.021.00_t109.210.33.021.00-everytime-couple-gold-stainless-steel-strap-179596


In [57]:
df

,price,cat,dilihat,dibeli,ulasan,reward_point,link,rating,Model No,Brand,...,Basic rest timer (up rom 0),"""Repeat on"" rest timer",Esports Activity,STR3AMUP! PC TOOL FOR STREAMING BIOMETRICS,Dual Gripc Coordinates,Light,Case Length in Milimeter,Text response/reject phone call with text (Android™ only),Control smartphone music,Preloaded gym activity profiles
0,Rp4.844.000,Pria,4.2 Rb,100,52 ulasan,24.220,https://www.jamtangan.com/p/seiko-prospex-srpd...,4.9,SRPD11K1,Seiko,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp1.430.000,Pria,1.1 Rb,85,6 ulasan,7.150,https://www.jamtangan.com/p/alexandre-christie...,4.8,ACF-6565-MCREPBA,Alexandre Christie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp2.759.000,Pria,1.8 Rb,99,37 ulasan,13.795,https://www.jamtangan.com/p/casio-gshock-gravi...,5,GA-1100-2BDR,Casio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp2.960.000,Pria,1.6 Rb,81,35 ulasan,14.800,https://www.jamtangan.com/p/spinnaker-vintage-...,5,SP-5062-33,Spinnaker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp1.098.000,Pria,1.9 Rb,25,10 ulasan,5.490,https://www.jamtangan.com/p/expedition-modern-...,5,EXF-6823-MFLUBBU,Expedition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Rp7.799.000,Unisex,676,7,0 ulasan,38.995,https://www.jamtangan.com/p/garmin-forerunner-...,,010-01863-F0,GARMIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes (Spotify Premium Only),"[Strength Training, Cardio Training, Elliptica..."
0,Rp185.000,Unisex,2.1 Rb,372,114 ulasan,925,https://www.jamtangan.com/p/casio-general-f91w...,4.9,F-91WM-3ADF,Casio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp137.000,Unisex,1.2 Rb,1.1 Rb,61 ulasan,685,https://www.jamtangan.com/p/casio-general-lq13...,4.9,LQ-139BMV-7ELDF,Casio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Rp185.000,Unisex,3.5 Rb,988,183 ulasan,925,https://www.jamtangan.com/p/casio-general-f91w...,5,F-91WM-1BDF,Casio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df = df.reset_index(drop=True)

In [82]:
# Drop column with NaN value >= 500
mod_df = df.dropna( axis=1, thresh=500)

In [83]:
mod_df.shape

(1284, 27)

In [151]:
def transform(x):
    if 'Rb' in x:
        z = x.replace('Rb','')
        z = float(z) * 1000
    else:
        z = x
    return z

In [152]:
# Cleaning the data
mod_df['price'] = mod_df['price'].apply(lambda x: x.replace('Rp','').replace('.',''))
mod_df['price'] = mod_df['price'].astype('int')
mod_df['dilihat'] = mod_df['dilihat'].apply(transform)
mod_df['dilihat'] = mod_df['dilihat'].astype('int')
mod_df['dibeli'] = mod_df['dibeli'].apply(transform)
mod_df['dibeli'] = mod_df['dibeli'].astype('int')
mod_df['ulasan'] = mod_df['ulasan'].apply(lambda x: x.replace('ulasan', '').strip())
mod_df['reward_point'] = mod_df['price'] * 0.005
mod_df['reward_point'] = mod_df['reward_point'].astype('int')
mod_df['nama_produk'] = mod_df['link'].apply(lambda x: x.split('/')[-1].replace('-',' ').title())

In [154]:
mod_df

,price,cat,dilihat,dibeli,ulasan,reward_point,link,rating,Model No,Brand,...,Case Back,Case Material,Glass Material,Strap Material,Water Resistant,Weight after packing,What's Inside Box,Features,Movement,nama_produk
0,4844000,Pria,4200,100,52,24220,https://www.jamtangan.com/p/seiko-prospex-srpd...,4.9,SRPD11K1,Seiko,...,Screw Down Case Back,Stainless Steel with Super Hard Coating,Hardlex crystal,Stainless Steel,Divers Watch 200 Meters/20 ATM/20 BAR,approx. 1kg,['1x Seiko Prospex SRPD11K1 Turtle Save The Oc...,"['Diver Watch', 'Sport Watch', 'Underictional ...","['Automatic', 'Manual Winding']",Seiko Prospex Srpd11K1 Turtle Save The Ocean A...
1,1430000,Pria,1100,85,6,7150,https://www.jamtangan.com/p/alexandre-christie...,4.8,ACF-6565-MCREPBA,Alexandre Christie,...,Screw Down Case Back,Stainless Steel,Mineral crystal,Leather/Kulit,50 Meters/5 ATM/5 BAR,approx. 700 gram,['1x Alexandre Christie Chronograph AC 6565 MC...,"['Chronograph', 'Sport Watch']",['Quartz/Battery'],Alexandre Christie Chronograph Ac 6565 Mcrepba...
2,2759000,Pria,1800,99,37,13795,https://www.jamtangan.com/p/casio-gshock-gravi...,5.0,GA-1100-2BDR,Casio,...,Screw On Case Back,Stainless Steel,Mineral crystal,Rubber/Silicon/Resin/Karet,200 Meters/20 ATM/20 BAR,approx. 500g,['1x Casio G-Shock Gravitymaster GA-1100-2BDR ...,"['Sport Watch', 'Hand shift feature (Hands mov...","['Quartz/Battery', 'Digital Analog']",Casio Gshock Gravitymaster Ga11002Bdr Twin Sen...
3,2960000,Pria,1600,81,35,14800,https://www.jamtangan.com/p/spinnaker-vintage-...,5.0,SP-5062-33,Spinnaker,...,Screw Down Case Back,Stainless Steel,Sapphire crystal with anti-reflective coating,Stainless Steel,180 Meters/18 ATM/18 BAR,approx. 700 gram,['1x Spinnaker Vintage Bradner SP-5062-33 Men ...,['Sport Watch'],['Automatic'],Spinnaker Vintage Bradner Sp506233 Men Green D...
4,1098000,Pria,1900,25,10,5490,https://www.jamtangan.com/p/expedition-modern-...,5.0,EXF-6823-MFLUBBU,Expedition,...,Screw Down Case Back,Stainless Steel,Mineral crystal,Leather/Kulit,50 Meters/5 ATM/5 BAR,approx. 800 gram,['1x Expedition Modern Classic E 6823 MF LUBBU...,['Sport Watch'],"['Quartz/Battery', 'Analog']",Expedition Modern Classic E 6823 Mf Lubbu Men ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,7799000,Unisex,676,7,0,38995,https://www.jamtangan.com/p/garmin-forerunner-...,NaN,010-01863-F0,GARMIN,...,Screw Down Case Back,Polymer,Mineral crystal,Rubber/Silicon/Resin/Karet,50 Meters/5 ATM/5 BAR,approx. 800 gram,"['<td colspan=""2"">1x Garmin Forerunner 645 010...","['Sport Watch', 'Smartwatch', 'For all Garmin ...","['Digital', 'Quartz/Battery']",Garmin Forerunner 645 01001863F0 Music Digital...
1280,185000,Unisex,2100,372,114,925,https://www.jamtangan.com/p/casio-general-f91w...,4.9,F-91WM-3ADF,Casio,...,Screw Down Case Back,Plastic/Resin,Resin Glass/Spherical Glass,Rubber/Silicon/Resin/Karet,30 Meters/3 ATM/3 BAR,approx. 500g,['1x Casio General F-91WM-3ADF Digital Dial Bl...,"['Classical Watch', 'Sport Watch', 'Daily alar...","['Quartz/Battery', 'Quartz Movement']",Casio General F91Wm3Adf Digital Dial Black Res...
1281,137000,Unisex,1200,1100,61,685,https://www.jamtangan.com/p/casio-general-lq13...,4.9,LQ-139BMV-7ELDF,Casio,...,Screw Down Case Back,Stainless Steel,Mineral crystal,Rubber/Silicon/Resin/Karet,Water Resistant,approx. 500g,['1x Casio General LQ-139BMV-7ELDF - Quartz - ...,['Classical Watch'],"['Quartz/Battery', 'Battery']",Casio General Lq139Bmv7Eldf Quartz Rubber Band...
1282,185000,Unisex,3500,988,183,925,https://www.jamtangan.com/p/casio-general-f91w...,5.0,F-91WM-1BDF,Casio,...,Screw Down Case Back,Plastic/Resin,Resin Glass/Spherical Glass,Rubber/Silicon/Resin/Karet,30 Meters/3 ATM/3 BAR,approx. 500g,['1x Casio General F-91WM-1BDF Digital Dial Bl...,"['Classical Watch', 'Daily alarm', 'Hourly tim...","['Quartz/Battery', 'Quartz Movement']",Casio General F91Wm1Bdf Digital Dial Black Res...


In [156]:
# Save to csv and xlsx format
mod_df.to_excel('full_clean_excel.xlsx', index=False)
mod_df.to_csv('full_clean2.csv', index=False)